# Predict Liver Failure based on People's Demographics

***(ETL Notebook)***

## 1. Introduction

Liver is the largest organ in the body of a Human being and other vertebrates, detoxifies various metabolites, synthesizes proteins, and produces biochemicals necessary for digestion. Its other roles in metabolism include the regulation of glycogen storage, decomposition of red blood cells and the production of hormones.

Liver Failure is a loss of liver function that could occur slowly (Chronic Liver Failure) or rapidly (Acute Liver Failure). There is no known way to compensate the absence of the liver function in the long term which makes Liver Failure a serious disease to worry about.

The commonly known causes for liver failure are paracetamol overdose, excessive alcohol consumption, viral hepatitis, food poisoning etc.

## 2. Data

### 2.1. Data Collection

As part of this Data Science project, our goal is to try and see if we can predict the possibility of a liver failure given the demographics of people. 

For this purpose, we will be using demographics and health information of people which is made available to us by the JPAC Center for Health Diagnosis and Control. Since 1990, they have conducted nationwide surveys using trained personnel and have collected a wide variety of demographic and health information using direct interviews, examinations, and blood samples. This dataset consists of selected information from 8,785 adults 20 years of age or older taken from the 2008–2009 and 2014–2015 surveys.

Our Goal is to try and use this data and build a Machine Learning Model that will help us in predicting the possibility of a liver failure in individuals using their demographics and health information.

Let us start by downloading this dataset which is made available to use on Kaggle and then transform this to a pandas dataframe for ease of data exploration.

In [ ]:
# import required packages and libraries
import types
import pandas as pd
import numpy as np

from ibm_botocore.client import Config
import ibm_boto3
import ibmos2spark

from pyspark.mllib.stat import Statistics

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190405072400-0000
KERNEL_ID = c4338e44-5df8-48c9-a24d-f84d6c8ee9cd


In [ ]:
def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_85bf8a7fa4e54387abd3bbb49b9490af = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='yR6pr44dLxKcEe_-J-YBRKtI9LaoOcG9v_c2zK_I1epP',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_85bf8a7fa4e54387abd3bbb49b9490af.get_object(Bucket='fundamentalsofscalabledatascience-donotdelete-pr-qbkdskud4vsck0',Key='ALF_Data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data = pd.read_csv(body)
df_data.head()

,Age,Gender,Region,Weight,Height,Body Mass Index,Obesity,Waist,Maximum Blood Pressure,Minimum Blood Pressure,...,PoorVision,Alcohol Consumption,HyperTension,Family HyperTension,Diabetes,Family Diabetes,Hepatitis,Family Hepatitis,Chronic Fatigue,ALF
0,65,M,east,56.0,162.1,21.31,0.0,83.6,135.0,71.0,...,0.0,1,0.0,0,0.0,1,1.0,0.0,0.0,0.0
1,36,M,south,60.2,162.2,22.88,0.0,76.6,96.0,52.0,...,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
2,66,M,east,83.9,162.5,31.77,1.0,113.2,115.0,57.0,...,0.0,1,0.0,0,1.0,0,0.0,0.0,0.0,0.0
3,54,M,east,69.4,160.5,26.94,0.0,77.9,110.0,57.0,...,0.0,1,0.0,0,0.0,0,0.0,0.0,0.0,0.0
4,63,M,north,73.1,159.2,28.84,0.0,89.3,132.0,73.0,...,0.0,0,1.0,0,0.0,0,0.0,0.0,0.0,0.0


Let us check the size of this dataset.

In [ ]:
df_data.shape

(8785, 30)

This dataset has data about 8785 people. There are 29 features which are made available and there is one Target Variable / Label which will tell us if the person has or is prone to a Liver Failure.

### 2.2. Exploratory Data Analysis

Now let us explore the dataset further to understand the features made available to us, identify any potential data quality issues and identify the columns that we will be using for creating out Machine Learning model which will help predict Liver Failures.

Let us start by taking a look at the columns and the data.

In [ ]:
df_data.columns

Index(['Age', 'Gender', 'Region', 'Weight', 'Height', 'Body Mass Index',
       'Obesity', 'Waist', 'Maximum Blood Pressure', 'Minimum Blood Pressure',
       'Good Cholesterol', 'Bad Cholesterol', 'Total Cholesterol',
       'Dyslipidemia', 'PVD', 'Physical Activity', 'Education', 'Unmarried',
       'Income', 'Source of Care', 'PoorVision', 'Alcohol Consumption',
       'HyperTension', 'Family  HyperTension', 'Diabetes', 'Family Diabetes',
       'Hepatitis', 'Family Hepatitis', 'Chronic Fatigue', 'ALF'],
      dtype='object')

In [ ]:
df_data.count(axis=0)

Age                       8785
Gender                    8785
Region                    8785
Weight                    8591
Height                    8594
Body Mass Index           8495
Obesity                   8495
Waist                     8471
Maximum Blood Pressure    8481
Minimum Blood Pressure    8409
Good Cholesterol          8768
Bad Cholesterol           8767
Total Cholesterol         8769
Dyslipidemia              8785
PVD                       8785
Physical Activity         8775
Education                 8765
Unmarried                 8333
Income                    7624
Source of Care            8785
PoorVision                8222
Alcohol Consumption       8785
HyperTension              8705
Family  HyperTension      8785
Diabetes                  8783
Family Diabetes           8785
Hepatitis                 8763
Family Hepatitis          8779
Chronic Fatigue           8750
ALF                       6000
dtype: int64

Based on the above list of columns and the data count in each we can observe the following:
* Some of the Column names have spaces in them
* There is some missing data.  

Let us first update the column names to remove the spaces in them.

In [ ]:
df_data.rename(index = str, columns = {'Body Mass Index':'BodyMassIndex', 'Maximum Blood Pressure':'MaximumBloodPressure',
                                       'Minimum Blood Pressure':'MinimumBloodPressure', 'Good Cholesterol':'GoodCholesterol',
                                       'Bad Cholesterol':'BadCholesterol', 'Total Cholesterol':'TotalCholesterol',
                                       'Physical Activity':'PhysicalActivity', 'Source of Care':'SourceOfCare',
                                       'Alcohol Consumption':'AlcoholConsumption', 'Family  HyperTension':'FamilyHyperTension',
                                       'Family Diabetes':'FamilyDiabetes', 'Family Hepatitis':'FamilyHepatitis',
                                       'Chronic Fatigue':'ChronicFatigue'}, inplace = True)

Now let us analyze the missing data and see how to address this.

The first column is "ALF" which is the target variable / Label that we will be predict. This column has values only for 6000 rows. This means that we will need to remove the remaining 2785 columns from the dataset as this data cannot be used for training our model. We can save these 2785 records in a different dataset and try predicting them at a later point in time using our machine learning model.

In [ ]:
df_data = df_data[df_data.ALF.notnull()]
df_data.count(axis=0)

Age                     6000
Gender                  6000
Region                  6000
Weight                  5867
Height                  5861
BodyMassIndex           5794
Obesity                 5794
Waist                   5785
MaximumBloodPressure    5794
MinimumBloodPressure    5748
GoodCholesterol         5992
BadCholesterol          5992
TotalCholesterol        5994
Dyslipidemia            6000
PVD                     6000
PhysicalActivity        5992
Education               5985
Unmarried               5699
Income                  5208
SourceOfCare            6000
PoorVision              5624
AlcoholConsumption      6000
HyperTension            5947
FamilyHyperTension      6000
Diabetes                5999
FamilyDiabetes          6000
Hepatitis               5987
FamilyHepatitis         5997
ChronicFatigue          5974
ALF                     6000
dtype: int64

Now we still have data missing for the columns BMI, Waist, Maximum Blood Pressue, Minimum Blood Pressure and Good/Bad/Total Cholestrol. This missing data constitutes for about 8% of our dataset. Since this missing data constitues for a small percentage if our data set, let us remove these rows instead of assuming their values.

In [ ]:
df_data = df_data[df_data.BodyMassIndex.notnull()]
df_data = df_data[df_data.MaximumBloodPressure.notnull()]
df_data = df_data[df_data.MinimumBloodPressure.notnull()]
df_data = df_data[df_data.Waist.notnull()]
df_data = df_data[df_data.GoodCholesterol.notna()]
df_data.count(axis=0)

Age                     5521
Gender                  5521
Region                  5521
Weight                  5521
Height                  5521
BodyMassIndex           5521
Obesity                 5521
Waist                   5521
MaximumBloodPressure    5521
MinimumBloodPressure    5521
GoodCholesterol         5521
BadCholesterol          5521
TotalCholesterol        5521
Dyslipidemia            5521
PVD                     5521
PhysicalActivity        5513
Education               5511
Unmarried               5233
Income                  4801
SourceOfCare            5521
PoorVision              5294
AlcoholConsumption      5521
HyperTension            5475
FamilyHyperTension      5521
Diabetes                5520
FamilyDiabetes          5521
Hepatitis               5510
FamilyHepatitis         5518
ChronicFatigue          5500
ALF                     5521
dtype: int64

Let us now look at the remaining missing values. The next feature for which data is missing is Physical activity. Let us analyze this further.

In [ ]:
df_data[df_data.PhysicalActivity.isna()].Age

1283    84
1401    48
3999    77
4015    84
4402    75
4650    69
4817    85
5503    68
Name: Age, dtype: int64

In [ ]:
df_data[df_data[['PhysicalActivity', 'Age']].Age>47][['PhysicalActivity', 'Age']] \
.groupby('PhysicalActivity').count()

,Age
PhysicalActivity,
1.0,711
2.0,1511
3.0,332
4.0,96


Based on the above data we can observe the following:
1. Physical Activity data is missing for 8 people. 
    * 7 people have ages > 65
    * 1 person has age = 48
2. Most of the people with age > 47 (~58%) have a Physical Activity value of (2.0).

So we can safely assume the missing physical activity values to be 2.0.

In [ ]:
df_data['PhysicalActivity'] = df_data['PhysicalActivity'].replace(np.NaN, 2.0)
df_data.count(axis = 0)

Age                     5521
Gender                  5521
Region                  5521
Weight                  5521
Height                  5521
BodyMassIndex           5521
Obesity                 5521
Waist                   5521
MaximumBloodPressure    5521
MinimumBloodPressure    5521
GoodCholesterol         5521
BadCholesterol          5521
TotalCholesterol        5521
Dyslipidemia            5521
PVD                     5521
PhysicalActivity        5521
Education               5511
Unmarried               5233
Income                  4801
SourceOfCare            5521
PoorVision              5294
AlcoholConsumption      5521
HyperTension            5475
FamilyHyperTension      5521
Diabetes                5520
FamilyDiabetes          5521
Hepatitis               5510
FamilyHepatitis         5518
ChronicFatigue          5500
ALF                     5521
dtype: int64

From the above we can see that we have data missing for the columns **Education, Unmarried, Income.** But we can clearly say that these columns do not have any direct bearing in predicting the Liver failure of an Individual. So we can directly get rid of these columns. In addition to these columns we can also get rid of the column **Source Of Care** as this column also does not have any direct bearing in predicting the Liver failure of an individual.

Now in addition to the above 3 columns we have the following data missing 
* 46 records missing Hypertension data
* Additionl 20 records missing Chronic Fatigue data
* Additional 9 records missing Hepatitis data
* Additional 3 records missing Family Hepatitis data
* Additional 222 records missing Poor Vision data.

To address the above, let us first start by removing the 4 columns identified above i.e. Education, Unmarried, Income and Source of care from our dataframe.

In [ ]:
df_data.drop(['Income', 'Unmarried', 'Education', 'SourceOfCare'], axis = 1, inplace = True)
df_data.count(axis=0)

Age                     5521
Gender                  5521
Region                  5521
Weight                  5521
Height                  5521
BodyMassIndex           5521
Obesity                 5521
Waist                   5521
MaximumBloodPressure    5521
MinimumBloodPressure    5521
GoodCholesterol         5521
BadCholesterol          5521
TotalCholesterol        5521
Dyslipidemia            5521
PVD                     5521
PhysicalActivity        5521
PoorVision              5294
AlcoholConsumption      5521
HyperTension            5475
FamilyHyperTension      5521
Diabetes                5520
FamilyDiabetes          5521
Hepatitis               5510
FamilyHepatitis         5518
ChronicFatigue          5500
ALF                     5521
dtype: int64

Now, regarding the missing data for "Hypertension", "Chronic Fatigue", "Hepatitis", "Family Hepatitis", "Poor Vision" - We do not have a way of getting these data values and also cannot assume these values. But the good part is that these constitute a small percentage (~5%) of our dataset. So let us go ahead and delete the corresponding records from our dataset.

In [ ]:
df_data = df_data[df_data.HyperTension.notna()]
df_data = df_data[df_data.ChronicFatigue.notna()]
df_data = df_data[df_data.Hepatitis.notna()]
df_data = df_data[df_data.FamilyHepatitis.notna()]
df_data = df_data[df_data.PoorVision.notna()]

Now we have cleaned up the data to address the data quality issues that we have seen so far. However there is one more thing that we need to do. The columns 'Gender' and 'Region' have non-numeric values. Let us convert these to numeric values as well before proceeding further.

In [ ]:
df_data['Gender'] = df_data['Gender'].replace(['M', 'F'],[1, 2])
df_data['Region'] = df_data['Region'].replace(['east', 'north', 'south', 'west'],[1, 2, 3, 4])
df_data.count(axis=0)

Age                     5221
Gender                  5221
Region                  5221
Weight                  5221
Height                  5221
BodyMassIndex           5221
Obesity                 5221
Waist                   5221
MaximumBloodPressure    5221
MinimumBloodPressure    5221
GoodCholesterol         5221
BadCholesterol          5221
TotalCholesterol        5221
Dyslipidemia            5221
PVD                     5221
PhysicalActivity        5221
PoorVision              5221
AlcoholConsumption      5221
HyperTension            5221
FamilyHyperTension      5221
Diabetes                5221
FamilyDiabetes          5221
Hepatitis               5221
FamilyHepatitis         5221
ChronicFatigue          5221
ALF                     5221
dtype: int64

In [ ]:
df_data.head()

,Age,Gender,Region,Weight,Height,BodyMassIndex,Obesity,Waist,MaximumBloodPressure,MinimumBloodPressure,...,PoorVision,AlcoholConsumption,HyperTension,FamilyHyperTension,Diabetes,FamilyDiabetes,Hepatitis,FamilyHepatitis,ChronicFatigue,ALF
0,65,1,1,56.0,162.1,21.31,0.0,83.6,135.0,71.0,...,0.0,1,0.0,0,0.0,1,1.0,0.0,0.0,0.0
1,36,1,3,60.2,162.2,22.88,0.0,76.6,96.0,52.0,...,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
2,66,1,1,83.9,162.5,31.77,1.0,113.2,115.0,57.0,...,0.0,1,0.0,0,1.0,0,0.0,0.0,0.0,0.0
3,54,1,1,69.4,160.5,26.94,0.0,77.9,110.0,57.0,...,0.0,1,0.0,0,0.0,0,0.0,0.0,0.0,0.0
4,63,1,2,73.1,159.2,28.84,0.0,89.3,132.0,73.0,...,0.0,0,1.0,0,0.0,0,0.0,0.0,0.0,0.0


#### Correlation Matrix

Now that we have addressed the data quality issues seen so far, as a next step let us generate a Correlation Matrix to assess the feature quality and the correlation between the features.

In [ ]:
# Generate Correlation Matrix
sparkDF = sqlContext.createDataFrame(df_data)
sparkRDD = sparkDF.rdd.map(list)
Statistics.corr(sparkRDD)

array([[ 1.        ,  0.03607417, -0.14561735, -0.04018436, -0.14240399,
         0.02931866,  0.00386674,  0.17627244,  0.55441277,  0.03732101,
         0.03677856,  0.14735726,  0.16229496,  0.02662777,  0.22899108,
        -0.14878635,  0.14437147,  0.20034582,  0.48344438, -0.10903974,
         0.24261645,  0.10794128,  0.23744657, -0.03501141,  0.16620354,
         0.35161475],
       [ 0.03607417,  1.        , -0.01371887,  0.28733144,  0.67224906,
        -0.05243305, -0.08545046,  0.1657609 ,  0.04312246,  0.17345687,
        -0.33126861,  0.03814234, -0.08550065,  0.09538984, -0.00261827,
         0.14511794,  0.03186122,  0.16016085, -0.0080201 , -0.05787393,
         0.02848511, -0.02168815,  0.06418898, -0.09968262,  0.01892975,
        -0.0270093 ],
       [-0.14561735, -0.01371887,  1.        , -0.08539687, -0.24991445,
         0.04353438,  0.03020844, -0.02252818, -0.01341035,  0.01312561,
        -0.05768014, -0.00110583, -0.02269628, -0.01306387, -0.0367307 ,
       

From the above correlation matrix we can see that there is no correlation between the columns. This could mean 2 things:
1. Either these columns contain a lot of information, OR
2. The data in these columns are irrelevant

Now let us perform some manual analysis of these features to see if all of these are relevant for our purpose.

On further review of the features, we can say that we can safely remove the following columns from the dataset:
* Region
    * We will not need the region data as we are looking for features related to individuals for our prediction model.
* Weight & Height
    * We have the "Body Mass Index" column which is derived as a function of the individual's height and weight and hence can safely get rid of these columns.
* Obesity
    * Obesity can be inferred from the "Body Mass Index" of an individual and hence we can also get rid of this column.

So, let us go ahead and remove these columns from our dataset.

In [ ]:
df_data.drop(['Region', 'Weight', 'Height', 'Obesity'], axis = 1, inplace = True)
print (df_data.shape)
df_data.head()

(5221, 22)


,Age,Gender,BodyMassIndex,Waist,MaximumBloodPressure,MinimumBloodPressure,GoodCholesterol,BadCholesterol,TotalCholesterol,Dyslipidemia,...,PoorVision,AlcoholConsumption,HyperTension,FamilyHyperTension,Diabetes,FamilyDiabetes,Hepatitis,FamilyHepatitis,ChronicFatigue,ALF
0,65,1,21.31,83.6,135.0,71.0,48.0,249.0,297.0,0,...,0.0,1,0.0,0,0.0,1,1.0,0.0,0.0,0.0
1,36,1,22.88,76.6,96.0,52.0,31.0,135.0,166.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
2,66,1,31.77,113.2,115.0,57.0,44.0,211.0,255.0,1,...,0.0,1,0.0,0,1.0,0,0.0,0.0,0.0,0.0
3,54,1,26.94,77.9,110.0,57.0,74.0,156.0,230.0,0,...,0.0,1,0.0,0,0.0,0,0.0,0.0,0.0,0.0
4,63,1,28.84,89.3,132.0,73.0,67.0,154.0,221.0,0,...,0.0,0,1.0,0,0.0,0,0.0,0.0,0.0,0.0


Now that we have performed some Initial Data Exploration and addressed the Data Quality issues found, let us go ahead and load this data into the IBM Object store.

### 2.3. Store Data in IBM Object Store

Let us set the credentials and connect to the IBM Data store for persisting our data.

In [ ]:
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'yR6pr44dLxKcEe_-J-YBRKtI9LaoOcG9v_c2zK_I1epP',
    'service_id': 'iam-ServiceId-dd08a5f3-28d2-4f87-bc12-4ec0662689f2',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token'}

configuration_name = 'os_85bf8a7fa4e54387abd3bbb49b9490af_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sparkDF = sqlContext.createDataFrame(df_data)
sparkDF = sparkDF.repartition(1)
sparkDF.write.parquet(cos.url('ALF_Data.parquet', 'fundamentalsofscalabledatascience-donotdelete-pr-qbkdskud4vsck0'))

Now that the data has been persisted in the IBM Object store, let us check and confirm that the data persisted is looking good.

In [ ]:
df_persisted_data = spark.read.parquet(cos.url('ALF_Data.parquet', 
                                               'fundamentalsofscalabledatascience-donotdelete-pr-qbkdskud4vsck0'))
df_persisted_data.show()

+---+------+-------------+-----+--------------------+--------------------+---------------+--------------+----------------+------------+---+----------------+----------+------------------+------------+------------------+--------+--------------+---------+---------------+--------------+---+
|Age|Gender|BodyMassIndex|Waist|MaximumBloodPressure|MinimumBloodPressure|GoodCholesterol|BadCholesterol|TotalCholesterol|Dyslipidemia|PVD|PhysicalActivity|PoorVision|AlcoholConsumption|HyperTension|FamilyHyperTension|Diabetes|FamilyDiabetes|Hepatitis|FamilyHepatitis|ChronicFatigue|ALF|
+---+------+-------------+-----+--------------------+--------------------+---------------+--------------+----------------+------------+---+----------------+----------+------------------+------------+------------------+--------+--------------+---------+---------------+--------------+---+
| 65|     1|        21.31| 83.6|               135.0|                71.0|           48.0|         249.0|           297.0|           0| 

The data persisted in the Object store is looking good. Going forward, we can use the data from the IBM Object store for further processing during the course of this project.